In [1]:
import os
print(os.getcwd())

d:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction


## Alternative method using donwsampled PNG by PNG for feature extraction

### Feature extraction with fivecrop function

In [ ]:
from huggingface_hub import login
from transformers import AutoModel
from PIL import Image
import torch
import numpy as np
from torchvision import transforms
from typing import Union, List
import os

class ConchFeatureExtractorFiveCrop:
    def __init__(self, token: str = ""):
        login(token=token)
        self.model = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
        self.conch, self.eval_transform = self.model.return_conch()

        # Add FiveCrop wrapper
        self.fivecrop_transform = transforms.Compose([
            transforms.FiveCrop(256),  # yields tuple of 5 images
            transforms.Lambda(lambda crops: torch.stack([self.eval_transform(c) for c in crops]))
        ])

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conch.to(self.device)
        self.conch.eval()

    def extract_fivecrop_features(self, image: Union[str, Image.Image]) -> torch.Tensor:
        """
        Extract 5xD features from a single image using FiveCrop.
        Returns a tensor of shape (5, D).
        """
        if isinstance(image, str):
            if not os.path.exists(image):
                raise FileNotFoundError(f"Image not found: {image}")
            image = Image.open(image).convert("RGB")
        elif isinstance(image, Image.Image):
            image = image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image type: {type(image)}")

        crops_tensor = self.fivecrop_transform(image)  # shape: (5, C, H, W)
        crops_tensor = crops_tensor.to(self.device)

        with torch.no_grad():
            features = self.conch(crops_tensor)  # shape: (5, D)

        return features.cpu()  # return to CPU for saving

    # def extract_fivecrop_features(self, image_paths: List[str], batch_size: int = 32) -> torch.Tensor:
    #     """
    #     Batch inference over multiple images using FiveCrop. Much faster.
    #     """
    #     all_features = []
    #     for i in range(0, len(image_paths), batch_size):
    #         batch_paths = image_paths[i:i+batch_size]
    #         batch_images = []

    #         for path in batch_paths:
    #             img = Image.open(path).convert("RGB")
    #             crops = self.fivecrop_transform(img)  # shape: (5, C, H, W)
    #             batch_images.append(crops)

    #         # Shape: (B, 5, C, H, W) → (B*5, C, H, W)
    #         batch_tensor = torch.stack(batch_images).to(self.device)  # (B, 5, C, H, W)
    #         B, N_CROPS, C, H, W = batch_tensor.shape
    #         batch_tensor = batch_tensor.view(B * N_CROPS, C, H, W)

    #         with torch.no_grad():
    #             features = self.conch(batch_tensor)  # shape: (B * 5, D)

    #         # Reshape back to (B, 5, D)
    #         features = features.view(B, N_CROPS, -1).cpu()
    #         all_features.append(features)

    #     return torch.cat(all_features, dim=0)  # shape: (N, 5, D)

    # def extract_fivecrop_features(self, image_paths: List[str]) -> torch.Tensor:
    #     """
    #     Efficient batch inference with FiveCrop.
    #     Returns shape: (B, 5, D)
    #     """
    #     images = []
    #     for path in image_paths:
    #         img = Image.open(path).convert("RGB")
    #         crops = transforms.FiveCrop(224)(img)
    #         crops = [self.eval_transform(c) for c in crops]  # (5, C, H, W)
    #         images.extend(crops)  # Flatten all crops directly

    #     # Now images: (B * 5, C, H, W)
    #     batch_tensor = torch.stack(images).to(self.device)  # shape: (B*5, C, H, W)

    #     with torch.no_grad():
    #         features = self.conch(batch_tensor).view(len(image_paths), 5, -1)  # (B, 5, D)

    #     return features.cpu().half()  # Reduce precision for size



    # def extract_fivecrop_features(self, image: Union[str, Image.Image]) -> torch.Tensor:
    #     """
    #     Extract 5xD features using efficient batched inference.
    #     Returns a tensor of shape (5, D).
    #     """
    #     import time
    #     start_total = time.perf_counter()

    #     if isinstance(image, str):
    #         if not os.path.exists(image):
    #             raise FileNotFoundError(f"Image not found: {image}")
    #         image = Image.open(image).convert("RGB")
    #     elif isinstance(image, Image.Image):
    #         image = image.convert("RGB")
    #     else:
    #         raise ValueError(f"Unsupported image type: {type(image)}")

    #     # FiveCrop
    #     t1 = time.perf_counter()
    #     crops = transforms.FiveCrop(224)(image)
    #     transformed_crops = [self.eval_transform(c) for c in crops]  # list of tensors
    #     t2 = time.perf_counter()
    #     print(f"⏱️ FiveCrop + Transform time: {t2 - t1:.4f} sec")

    #     # Efficient batch inference
    #     t1 = time.perf_counter()
    #     input_tensor = torch.stack(transformed_crops).to(self.device)  # (5, C, H, W)
    #     with torch.no_grad():
    #         features = self.conch(input_tensor)  # (5, D)
    #     t2 = time.perf_counter()
    #     print(f"⏱️ Inference time (batched): {t2 - t1:.4f} sec")

    #     print(f"✅ Total time: {time.perf_counter() - start_total:.4f} sec\n")
    #     return features.cpu().half()


    # def extract_fivecrop_features(self, image: Union[str, Image.Image]) -> torch.Tensor:
    #     """
    #     Extract 5xD features using FiveCrop in parallel using CUDA streams.
    #     Returns a tensor of shape (5, D), and prints time breakdown.
    #     """
    #     start_total = time.perf_counter()

    #     # === Load Image ===
    #     start = time.perf_counter()
    #     if isinstance(image, str):
    #         if not os.path.exists(image):
    #             raise FileNotFoundError(f"Image not found: {image}")
    #         image = Image.open(image).convert("RGB")
    #     elif isinstance(image, Image.Image):
    #         image = image.convert("RGB")
    #     else:
    #         raise ValueError(f"Unsupported image type: {type(image)}")
    #     end = time.perf_counter()
    #     print(f"⏱️ Image load time: {end - start:.4f} sec")

    #     # === FiveCrop ===
    #     start = time.perf_counter()
    #     crops = transforms.FiveCrop(224)(image)
    #     end = time.perf_counter()
    #     print(f"⏱️ FiveCrop time: {end - start:.4f} sec")

    #     # === Transform each crop ===
    #     start = time.perf_counter()
    #     transformed_crops = [self.eval_transform(c) for c in crops]
    #     end = time.perf_counter()
    #     print(f"⏱️ Transform time: {end - start:.4f} sec")

    #     # === Parallel CUDA inference ===
    #     def run_inference(crop_tensor, stream):
    #         with torch.cuda.stream(stream):
    #             input_tensor = crop_tensor.unsqueeze(0).to(self.device)
    #             with torch.no_grad():
    #                 output = self.conch(input_tensor)  # shape: (1, D)
    #             return output.squeeze(0).cpu()

    #     start = time.perf_counter()
    #     streams = [torch.cuda.Stream() for _ in range(5)]
    #     with ThreadPoolExecutor(max_workers=5) as executor:
    #         futures = [
    #             executor.submit(run_inference, transformed_crops[i], streams[i])
    #             for i in range(5)
    #         ]
    #         features = [f.result() for f in futures]
    #     torch.cuda.synchronize()
    #     end = time.perf_counter()
    #     print(f"⏱️ Inference time (parallel): {end - start:.4f} sec")

    #     # === Final stack ===
    #     start = time.perf_counter()
    #     final_output = torch.stack(features)
    #     end = time.perf_counter()
    #     print(f"⏱️ Tensor stack time: {end - start:.4f} sec")

    #     print(f"✅ Total time for image: {time.perf_counter() - start_total:.4f} sec\n")

    #     return final_output

    
    # def extract_features_batch(self, image_paths: List[str], batch_size: int = 32) -> torch.Tensor:
    #     """
    #     Extract FiveCrop features for a batch of image paths.
    #     Returns a tensor of shape (N, 5, D), where N is batch size.
    #     """
    #     images = []
    #     for path in image_paths:
    #         img = Image.open(path).convert("RGB")
    #         crops = self.fivecrop_transform(img)  # (5, C, H, W)
    #         images.append(crops)  # append 5-crop tensor

    #     # Shape: (N, 5, C, H, W)
    #     batch_tensor = torch.stack(images).to(self.device)  # shape: (B, 5, C, H, W)
    #     B, N_CROPS, C, H, W = batch_tensor.shape
    #     batch_tensor = batch_tensor.view(B * N_CROPS, C, H, W)

    #     with torch.no_grad():
    #         features = self.conch(batch_tensor)  # (B * 5, D)

    #     features = features.view(B, N_CROPS, -1)  # (B, 5, D)
    #     return features.cpu()

## Test for one folder

In [3]:
import os
import torch
from PIL import Image
from tqdm import tqdm

# Assuming your ConchFeatureExtractorFiveCrop class is already defined above

def extract_features_from_png_folder(
    image_folder: str,
    output_folder: str,
    extractor: ConchFeatureExtractorFiveCrop
):
    os.makedirs(output_folder, exist_ok=True)

    png_files = [f for f in os.listdir(image_folder) if f.lower().endswith('.png')]
    print(f"🔍 Found {len(png_files)} PNG files in {image_folder}")

    for filename in tqdm(png_files, desc="Extracting features"):
        image_path = os.path.join(image_folder, filename)
        try:
            features = extractor.extract_fivecrop_features(image_path)  # (5, D)
            if features.shape[0] != 5:
                print(f"⚠️ Skipped {filename}: unexpected shape {features.shape}")
                continue

            save_path = os.path.join(output_folder, filename.replace('.png', '.pt'))
            torch.save(features.contiguous(), save_path, _use_new_zipfile_serialization=False)
        except Exception as e:
            print(f"❌ Failed on {filename}: {e}")

    print("✅ Feature extraction complete.")


# ==== Run This Section ====
if __name__ == "__main__":
    # Instantiate extractor
    extractor = ConchFeatureExtractorFiveCrop()

    # Define input/output
    image_folder = r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data\TCGA-EI-6507_MSIH"    # 🔁 CHANGE THIS
    output_folder = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Test"  # 🔁 CHANGE THIS

    # Run extraction
    extract_features_from_png_folder(image_folder, output_folder, extractor)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful
🔍 Found 3272 PNG files in D:\Aamir Gulzar\KSA_project2\dataset\patch_data\TCGA-EI-6507_MSIH


Extracting features:  32%|███▏      | 1056/3272 [02:01<04:15,  8.67it/s]


KeyboardInterrupt: 

In [ ]:
import os
import torch
from PIL import Image
from tqdm import tqdm

# Assuming your ConchFeatureExtractorFiveCrop class is already defined above

def extract_features_from_png_folder(
    image_folder: str,
    output_folder: str,
    extractor: ConchFeatureExtractorFiveCrop
):
    os.makedirs(output_folder, exist_ok=True)

    png_files = [f for f in os.listdir(image_folder) if f.lower().endswith('.png')]
    print(f"🔍 Found {len(png_files)} PNG files in {image_folder}")

    for filename in tqdm(png_files, desc="Extracting features"):
        image_path = os.path.join(image_folder, filename)
        try:
            features = extractor.extract_fivecrop_features(image_path)  # (5, D)
            if features.shape[0] != 5:
                print(f"⚠️ Skipped {filename}: unexpected shape {features.shape}")
                continue

            save_path = os.path.join(output_folder, filename.replace('.png', '.pt'))
            torch.save(features.contiguous(), save_path, _use_new_zipfile_serialization=False)
        except Exception as e:
            print(f"❌ Failed on {filename}: {e}")

    print("✅ Feature extraction complete.")


# ==== Run This Section ====
if __name__ == "__main__":
    # Instantiate extractor
    extractor = ConchFeatureExtractorFiveCrop()

    # Define input/output
    image_folder = "D:\Aamir Gulzar\KSA_project2\dataset\patch_data\TCGA-EI-6507_MSIH"    # 🔁 CHANGE THIS
    output_folder = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Test"  # 🔁 CHANGE THIS

    # Run extraction
    extract_features_from_png_folder(image_folder, output_folder, extractor)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful
🔍 Found 3272 PNG files in D:\Aamir Gulzar\KSA_project2\dataset\patch_data\TCGA-EI-6507_MSIH


Extracting features: 100%|██████████| 3272/3272 [00:00<00:00, 37544.22it/s]

❌ Failed on TCGA-EI-6507_MSIH_x0_y19456_patch00000.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y19968_patch00001.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y20480_patch00002.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y20992_patch00003.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y21504_patch00004.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y23552_patch00005.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x0_y24064_patch00006.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x10240_y10240_patch00512.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x10240_y10752_patch00513.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-EI-6507_MSIH_x10240_y11264_patch00514.png: [Errno 2] No such file or directory: 'D'
❌ Failed on TCGA-E

## Sirs method

In [14]:
model = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
eval_transform = model.return_conch()
fivecrop_transform = transforms.Compose([
            # transforms.Resize(256),
            transforms.FiveCrop(224),  # yields tuple of 5 images
            transforms.Lambda(lambda crops: torch.stack([eval_transform(c) for c in crops]))
])

# make dataloader for WSI patches
import os
from tqdm import tqdm

@torch.no_grad()
def extract_embeddings_patch_by_patch(model, dataloader, save_dir):
    """
    Extract and save embeddings for each WSI, patch by patch, without averaging the five crops.
    Args:
    - model: The model used to extract embeddings.
    - dataloader: Dataloader providing WSI patches and labels.
    - save_dir: Directory where the extracted embeddings will be saved.
    Returns:
    - None: The function saves the extracted embeddings to disk.
    """
    device = next(model.parameters()).device
    print(f'The size of input dataloader is {len(dataloader)}')

    for batch_idx, (images, labels) in tqdm(enumerate(dataloader), total=len(dataloader)):
        wsi_name = dataloader.dataset.slides[batch_idx]
        print(f"Processing WSI {batch_idx+1} {wsi_name}")
        # make a new directory for each WSI
        save_dir_wsi = os.path.join(save_dir, f'{wsi_name}')
        # Check if the WSI directory already exists, skip if processed
        if os.path.exists(save_dir_wsi):
            print(f"WSI {batch_idx+1} {wsi_name} already processed. Skipping...")
            continue
        os.makedirs(save_dir_wsi, exist_ok=True)
        batch_indices = dataloader.batch_sampler.indices[batch_idx]        
        for patch_idx, dataset_idx in enumerate(batch_indices):  # Loop through indices of patches for current WSI
            patch_name = os.path.splitext(os.path.basename(dataloader.dataset.tiles[dataset_idx]))[0]
            image = images[patch_idx]
            label = labels[patch_idx]
            image = image.permute(0, 2, 3, 1).cpu().numpy()  # Convert to (num_crops, H, W, C)
            image = (image * 255).astype(np.uint8)  # Convert back to uint8 format
            # Apply Normalization on Each Crop (Matching Virchow2 & Conch)
            image = torch.stack([fivecrop_transform(Image.fromarray(im)) for im in image])
            # Move image to the same device as the model
            image = image.to(device)
            # Extract embeddings
            with torch.inference_mode():
                # Extract embeddings for the chunk
                features = model(image)
            # Save embeddings to disk
            save_path = os.path.join(save_dir_wsi, f'{patch_name}.pt')
            # print(f"Saving features for patch {i + 1} to {save_path}")
            torch.save(features, save_path)
            # also save in simple text format in text file
            # save_path_txt = os.path.join(save_dir_wsi, f'{wsi_name}_{i}.txt')
            # np.savetxt(save_path_txt, features.cpu().numpy() , delimiter=",")

## Final Pipeline Updated

In [2]:
from tqdm import tqdm
import pandas as pd
import torch
import os
import glob

def run_full_pipeline_fivecrop(
    patch_data_dir: str,
    extractor,
    non_white_csv: str,
    output_root: str
):
    # Load metadata
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))

    # Iterate through all slides with tqdm
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))
    for slide_dir in tqdm(slide_dirs, desc="🧩 Processing slides", unit="slide", position=0):
        slide_name = os.path.basename(slide_dir)
        patch_feat_dir = os.path.join(output_root, slide_name)
        os.makedirs(patch_feat_dir, exist_ok=True)

        # Filter valid patches
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        valid_patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]

        if not valid_patch_paths:
            tqdm.write(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Get already processed patch names
        existing_feats = set(os.path.splitext(f.name)[0] for f in os.scandir(patch_feat_dir) if f.name.endswith(".pt"))

        # Filter out already processed patches
        to_process = [p for p in valid_patch_paths if os.path.splitext(os.path.basename(p))[0] not in existing_feats]

        if not to_process:
            tqdm.write(f"  ⏭️ Skipped: All {len(valid_patch_paths)} patches already processed for {slide_name}")
            continue

        # ▶️ Patch-level progress bar (with position=1)
        for patch_path in tqdm(to_process, desc=f"🔄 {slide_name}", unit="patch", position=1, leave=False):
            patch_name = os.path.splitext(os.path.basename(patch_path))[0]
            save_path = os.path.join(patch_feat_dir, patch_name + ".pt")

            try:
                features = extractor.extract_fivecrop_features(patch_path)  # (5, D)
                torch.save(features, save_path)
            except Exception as e:
                tqdm.write(f"  ❌ Failed to process patch {patch_path}: {e}")
                continue

        tqdm.write(f"  ✅ Done: Processed {len(to_process)} new patches (total: {len(valid_patch_paths)}) for {slide_name}")

In [5]:
extractor = ConchFeatureExtractorFiveCrop()
import glob

run_full_pipeline_fivecrop(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data",
    extractor=extractor,
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5_fivecrop_Updated"
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful


🧩 Processing slides:   0%|          | 1/417 [00:00<00:51,  8.12slide/s]

  ⏭️ Skipped: All 1878 patches already processed for TCGA-3L-AA1B_nonMSIH


🧩 Processing slides:   0%|          | 2/417 [00:00<01:53,  3.67slide/s]

  ⏭️ Skipped: All 4240 patches already processed for TCGA-4N-A93T_nonMSIH


...

  ⏭️ Skipped: All 4849 patches already processed for TCGA-QG-A5Z2_MSIH
  ⏭️ Skipped: All 4126 patches already processed for TCGA-QL-A97D_nonMSIH


🧩 Processing slides: 100%|█████████▉| 415/417 [02:32<00:00,  6.10slide/s]

  ⏭️ Skipped: All 5746 patches already processed for TCGA-SS-A7HO_nonMSIH


🧩 Processing slides: 100%|██████████| 417/417 [02:32<00:00,  2.73slide/s]


  ⏭️ Skipped: All 5462 patches already processed for TCGA-T9-A92H_nonMSIH
  ⏭️ Skipped: All 4532 patches already processed for TCGA-WS-AB45_MSIH
